### yahoofinance 크롤링

In [2]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service as EdgeService
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

In [12]:
# 티커 배열
tickers = ['SOXL']  # 예시로 몇 가지 티커 추가


# 크롬 웹드라이버 설정
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# 결과를 저장할 CSV 파일 설정
output_file = "news_results.csv"

# CSV 파일의 헤더 작성
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Ticker", "Date", "Title", "Link", "Content"])

# 뉴스 크롤링 함수
def crawl_news_for_ticker(ticker):
    url = f"https://finance.yahoo.com/quote/{ticker}/news/"
    driver.get(url)
    time.sleep(3)  # 페이지 로딩 대기

    # 뉴스 항목 추출
    try:
        news_items = driver.find_elements(By.XPATH, '//li[@class="stream-item story-item yf-1usaaz9"]')
        print(f"Ticker: {ticker} - 뉴스 제목 및 내용:")

        # 각 뉴스 기사에 대해 처리
        for item in news_items:
            # 뉴스 제목 추출
            title_element = item.find_element(By.XPATH, './/h3[@class="clamp  yf-1sxfjua"]')
            title = title_element.text
            print(f"\n- 제목: {title}")

            # 뉴스 기사 링크 추출
            link_element = item.find_element(By.XPATH, './/div[@class="content yf-1sxfjua"]//a[@class="subtle-link fin-size-small titles noUnderline yf-1e4diqp"]')
            news_link = link_element.get_attribute('href')
            print(f"링크: {news_link}")

            # 뉴스 기사 본문 및 작성일 크롤링
            driver.execute_script(f"window.open('{news_link}', '_blank');")
            driver.switch_to.window(driver.window_handles[1])

            time.sleep(3)  # 새 탭이 열리고 내용이 로드되도록 대기

            # 뉴스 내용 크롤링
            try:
                content = driver.find_element(By.XPATH, '//*[@id="nimbus-app"]/section/section/section/article/div/div[1]/div[3]/div[2]').text
                print(f"내용: {content[:500]}...")  # 내용이 길 경우 앞부분만 출력
            except:
                content = "내용을 가져오지 못했습니다."
                print(content)

            # 작성일 크롤링 및 datetime 형식 변환
            
            time.sleep(3)

            date_element = driver.find_element(By.XPATH, '//time[@class="byline-attr-meta-time"]')
            date_raw = date_element.get_attribute('datetime')  # 원본 날짜 값 가져오기
            date = datetime.strptime(date_raw, '%Y-%m-%dT%H:%M:%S.%fZ').strftime('%Y-%m-%d')  # datetime 형식 변환
            print(f"작성일: {date}")

            print(date)

            # 결과를 CSV 파일에 저장
            with open(output_file, mode='a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow([ticker, date, title, news_link, content])  # 내용 전체 저장

            driver.close()
            driver.switch_to.window(driver.window_handles[0])  # 원래 탭으로 돌아옴

    except Exception as e:
        print(f"Error retrieving news for {ticker}: {str(e)}")

# 각 티커에 대해 뉴스 제목 및 내용 크롤링
for ticker in tickers:
    crawl_news_for_ticker(ticker)

# 브라우저 종료
driver.quit()


KeyboardInterrupt: 